# Create Slice for Fabric Rack
This is the slice creation for the ```Webserver``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.
 

## 1. Set up the Experiment

In this section, you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Traffic Generation Assignment.
### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 3 nodes arranged in a line, with one node designated as a Server. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. The Server node will have 2 network cards (NICs), while each host node will have 1 NIC. All nodes will be preloaded with an 'Ubuntu' Linux OS. The Server node will be responsible for passing traffic on to each of the hosts connectedE. Upon submission, the slice will be named 'MyWebServerSlice'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "MyWebServerSlice"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(slice_name)
    
    # deleted spaces in the calls
    
    # Add node
    server = slice.add_node(name='Server', site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    ifaces1 = server.add_component(model='NIC_Basic', name='server-nic1').get_interfaces()[0] 
    ifaces2 = server.add_component(model='NIC_Basic', name='server-nic2').get_interfaces()[0] 
    
    # Add node
    node1 = slice.add_node(name='Client1', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    ifacen1 = node1.add_component(model='NIC_Basic', name='node1-nic1').get_interfaces()[0] 
    
    # Add node
    node2 = slice.add_node(name='Client2', site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    ifacen2 = node2.add_component(model='NIC_Basic', name='node2-nic1').get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name='bridge1', interfaces=[ifaces1, ifacen1])
    net2 = slice.add_l2network(name='bridge2', interfaces=[ifaces2, ifacen2])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "bridge1" & "bridge2".

This slice contains a single subnetwork in IPv4: 10.20.30.0/24 between server and each of the host clients, however it is split between the two l2networks "bridge1" & "bridge1". This slice will provide IP 10.20.30.41 to "Node_A" under "bridge1" & "10.20.30.41" under "bridge2", host 1 will obtain 10.20.30.42 and host2 will obtain 10.20.30.43 following the guidelines set by the network subnet. In this slice we use the alternative "cidr" to set the subnet rather than directly passing the subnet.

Since the two L2 networks use the same subnet, this will cause issues if the routes are left to be added on their own. In the following cell we modified the routes such that they use the specific ip of each node inside of the server routing table to avoid accidentally sending traffic to the incorrect node. After providing the IPs to the corresponding devices, the cell will output the network status and execute "ip a" to check the connections that are active.


In [ ]:
# Setup Network

try:
    server_name  = 'Server'
    node_1_name = 'Client1'
    node_2_name = 'Client2'

    network_service_name1 = "bridge1"
    network_service_name2 = "bridge2"

    server_ip1 = '10.20.30.40'
    server_ip2 = '10.20.30.41'
    node1_ip = '10.20.30.42'
    node2_ip = '10.20.30.43'
    
    server = slice.get_node(name=server_name)        
    ifaces1 = server.get_interface(network_name=network_service_name1)  
    ifaces1.set_ip(ip=server_ip1, cidr="24")
    
    ifaces2 = server.get_interface(network_name=network_service_name2)  
    ifaces2.set_ip(ip=server_ip2, cidr="24")
    
    Device_P1 = ifaces1.get_device_name()
    Device_P2 = ifaces2.get_device_name()
    
    #Node1 conection
    stdout, stderr = server.execute(f'ip addr show {ifaces1.get_os_interface()}')
    server.execute(f"sudo ip link set dev {Device_P1} up")
    print (stdout)
    #node2 conection
    stdout, stderr = server.execute(f'ip addr show {ifaces2.get_os_interface()}')
    server.execute(f"sudo ip link set dev { Device_P2 } up")
    print (stdout)
    
    node1 = slice.get_node(name=node_1_name)        
    ifacen1 = node1.get_interface(network_name=network_service_name1)  
    ifacen1.set_ip(ip=node1_ip, cidr="24")
    
    stdout, stderr = node1.execute(f'ip addr show {ifacen1.get_os_interface()}')
    node1.execute(f"sudo ip link set dev { ifacen1.get_device_name()} up")
    print (stdout)
    
    node2 = slice.get_node(name=node_2_name)        
    ifacen2 = node2.get_interface(network_name=network_service_name2)
    ifacen2.set_ip(ip=node2_ip, cidr="24")
    stdout, stderr = node2.execute(f'ip addr show {ifacen2.get_os_interface()}')
    node2.execute(f"sudo ip link set dev { ifacen2.get_device_name()} up")
    print (stdout)
    
    # 1 packets transmitted, 0 received, +1 errors, 100% packet loss, time 0ms
    # swamp the words "enp7s0" and "enp8s0" this just means that the ips were created on oposite devices
    
    server.execute(f'sudo route del -net 10.20.30.0 gw 0.0.0.0 netmask 255.255.255.0 dev {Device_P2}')
    server.execute(f'sudo route del -net 10.20.30.0 gw 0.0.0.0 netmask 255.255.255.0 dev {Device_P1}')
    server.execute(f'sudo ip route add {node1_ip} dev {Device_P1}')
    server.execute(f'sudo ip route add {node2_ip} dev {Device_P2}')
    print ("Testing Connection...")
    server.execute(f'ping 10.20.30.43 -c 1')
    server.execute(f'ping 10.20.30.42 -c 1')
    
except Exception as e:
    print(f"Error: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
we only need to install a couple of tools for our slice, the first of these will be "net-tools" this will help us monitor traffic inside of the nodes.

Additionally for this slice, we need to install some software that is only available from a GitHub repository. First we download the webpage example as a tar file, and then extract its contents. Once the previous step has been completed we run the configuration scripts on each machine to set up the machines with their corresponding tasks, whether it is to listen as a server or request information as a client.

In [ ]:
# Install Software
try:
    print("Installing tools testing tools")
    stdout, stderr = server.execute(f'sudo apt-get update', quiet=False)
    stdout, stderr = server.execute(f'sudo apt-get install net-tools', quiet=False)
    
    server = slice.get_node(name=server_name)        
    server.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    server.execute("tar xvfz webexample.tar.gz")

    server.execute("sudo ./webexample/websrv_init.sh")
    server.execute("sudo cp -avr webexample/website ../../var/www")
    
    #node 1
    node1.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    node1.execute("tar xvfz webexample.tar.gz")
    node1.execute("sudo ./webexample/webcl_init.sh")
    
    #node2    
    node2.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    node2.execute("tar xvfz webexample.tar.gz")
    node2.execute("sudo ./webexample/webcl_init.sh")

except Exception as e:
    print(f"Error: {e}")

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Webserver. You can either open it on the explorer or click [Here](./webserver.ipynb) to open the next notebook.